In [1]:
from tsfresh.feature_extraction import extract_features
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from tsfresh.feature_extraction.settings import from_columns

import numpy as np
import pandas as pd

/Users/mchrist/Documents/Research/tsfresh/venv/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


This notebooks illustrates the `"fc_parameters"` or `"kind_to_fc_parameters"` dictionaries.

For a detailed explanation, see also http://tsfresh.readthedocs.io/en/latest/text/feature_extraction_settings.html

## Construct a time series container

We construct the time series container that includes two sensor time series, _"temperature"_ and _"pressure"_, for two devices _"a"_ and _"b"_

In [2]:
df = pd.DataFrame({"id": ["a", "a", "b", "b"], "temperature": [1,2,3,1], "pressure": [-1, 2, -1, 7]})
df

,id,pressure,temperature
0,a,-1,1
1,a,2,2
2,b,-1,3
3,b,7,1


## The default_fc_parameters

Which features are calculated by tsfresh is controlled by a dictionary that contains a mapping from feature calculator names to their parameters. 
This dictionary is called `fc_parameters`. It maps feature calculator names (=keys) to parameters (=values). As keys, always the same names as in the tsfresh.feature_extraction.feature_calculators module are used.

In the following we load an exemplary dictionary

In [3]:
settings_minimal = MinimalFCParameters() # only a few basic features
settings_minimal

{'length': None,
 'maximum': None,
 'mean': None,
 'median': None,
 'minimum': None,
 'standard_deviation': None,
 'sum_values': None,
 'variance': None}

This dictionary can passed to the extract method, resulting in a few basic time series beeing calculated:

In [4]:
X_tsfresh = extract_features(df, column_id="id", default_fc_parameters = settings_minimal)
X_tsfresh.head()

Feature Extraction: 100%|██████████| 2/2 [00:00<00:00, 45.94it/s]


,pressure__sum_values,pressure__length,pressure__maximum,pressure__mean,pressure__median,pressure__minimum,pressure__standard_deviation,pressure__variance,temperature__minimum,temperature__standard_deviation,temperature__length,temperature__mean,temperature__sum_values,temperature__variance,temperature__median,temperature__maximum
id,,,,,,,,,,,,,,,,
a,1.0,2.0,2.0,0.5,0.5,-1.0,1.5,2.25,1.0,0.5,2.0,1.5,3.0,0.25,1.5,2.0
b,6.0,2.0,7.0,3.0,3.0,-1.0,4.0,16.00,1.0,1.0,2.0,2.0,4.0,1.00,2.0,3.0


By using the settings_minimal as value of the default_fc_parameters parameter, those settings are used for all type of time series. In this case, the `settings_minimal` dictionary is used for both _"temperature"_ and _"pressure"_ time series.

Now, lets say we want to remove the length feature and prevent it from beeing calculated. We just delete it from the dictionary.

In [5]:
del settings_minimal["length"]
settings_minimal

{'maximum': None,
 'mean': None,
 'median': None,
 'minimum': None,
 'standard_deviation': None,
 'sum_values': None,
 'variance': None}

Now, if we extract features for this reduced dictionary, the length feature will not be calculated

In [6]:
X_tsfresh = extract_features(df, column_id="id", default_fc_parameters = settings_minimal)
X_tsfresh.head()

Feature Extraction: 100%|██████████| 2/2 [00:00<00:00, 82.04it/s]


,pressure__sum_values,pressure__maximum,pressure__mean,pressure__median,pressure__minimum,pressure__standard_deviation,pressure__variance,temperature__minimum,temperature__variance,temperature__mean,temperature__sum_values,temperature__median,temperature__standard_deviation,temperature__maximum
id,,,,,,,,,,,,,,
a,1.0,2.0,0.5,0.5,-1.0,1.5,2.25,1.0,0.25,1.5,3.0,1.5,0.5,2.0
b,6.0,7.0,3.0,3.0,-1.0,4.0,16.00,1.0,1.00,2.0,4.0,2.0,1.0,3.0


## The kind_to_fc_parameters

now, lets say we do not want to calculate the same features for both type of time series. Instead there should be different sets of features for each kind.

To do that, we can use the `kind_to_fc_parameters` parameter, which lets us finely specifiy which `fc_parameters` we want to use for which kind of time series:

In [7]:
fc_parameters_pressure = {"length": None, 
                          "sum_values": None}

fc_parameters_temperature = {"maximum": None, 
                             "minimum": None}

kind_to_fc_parameters = {
    "temperature": fc_parameters_temperature,
    "pressure": fc_parameters_pressure
}

print(kind_to_fc_parameters)

{'pressure': {'length': None, 'sum_values': None}, 'temperature': {'minimum': None, 'maximum': None}}


So, in this case, for sensor _"pressure"_ both _"max"_ and _"min"_ are calculated. For the _"temperature"_ signal, the length and sum_values features are extracted instead.

In [8]:
X_tsfresh = extract_features(df, column_id="id", kind_to_fc_parameters = kind_to_fc_parameters)
X_tsfresh.head()

Feature Extraction: 100%|██████████| 2/2 [00:00<00:00, 128.97it/s]


,pressure__sum_values,pressure__length,temperature__minimum,temperature__maximum
id,,,,
a,1.0,2.0,1.0,2.0
b,6.0,2.0,1.0,3.0


So, lets say we lost the kind_to_fc_parameters dictionary. Or we apply a feature selection algorithm to drop 
irrelevant feature columns, so our extraction settings contain irrelevant features. 

In both cases, we can use the provided "from_columns" method to infer the creating dictionary from 
the dataframe containing the features

In [9]:
recovered_settings = from_columns(X_tsfresh)
recovered_settings

{'pressure': {'length': None, 'sum_values': None},
 'temperature': {'maximum': None, 'minimum': None}}

Lets drop a column to show that the inferred settings dictionary really changes

In [10]:
X_tsfresh.iloc[:, 1:]

,pressure__length,temperature__minimum,temperature__maximum
id,,,
a,2.0,1.0,2.0
b,2.0,1.0,3.0


In [11]:
recovered_settings = from_columns(X_tsfresh.iloc[:, 1:])
recovered_settings

{'pressure': {'length': None},
 'temperature': {'maximum': None, 'minimum': None}}

## More complex dictionaries

We provide custom fc_parameters dictionaries with greater sets of features.

The `EfficientFCParameters` contain features and parameters that should be calculated quite fastly:

In [12]:
settings_efficient = EfficientFCParameters()
settings_efficient

{'abs_energy': None,
 'absolute_sum_of_changes': None,
 'ar_coefficient': [{'coeff': 0, 'k': 10},
  {'coeff': 1, 'k': 10},
  {'coeff': 2, 'k': 10},
  {'coeff': 3, 'k': 10},
  {'coeff': 4, 'k': 10}],
 'augmented_dickey_fuller': None,
 'autocorrelation': [{'lag': 0},
  {'lag': 1},
  {'lag': 2},
  {'lag': 3},
  {'lag': 4},
  {'lag': 5},
  {'lag': 6},
  {'lag': 7},
  {'lag': 8},
  {'lag': 9}],
 'binned_entropy': [{'max_bins': 10}],
 'count_above_mean': None,
 'count_below_mean': None,
 'cwt_coefficients': [{'coeff': 0, 'w': 2, 'widths': (2, 5, 10, 20)},
  {'coeff': 0, 'w': 5, 'widths': (2, 5, 10, 20)},
  {'coeff': 0, 'w': 10, 'widths': (2, 5, 10, 20)},
  {'coeff': 0, 'w': 20, 'widths': (2, 5, 10, 20)},
  {'coeff': 1, 'w': 2, 'widths': (2, 5, 10, 20)},
  {'coeff': 1, 'w': 5, 'widths': (2, 5, 10, 20)},
  {'coeff': 1, 'w': 10, 'widths': (2, 5, 10, 20)},
  {'coeff': 1, 'w': 20, 'widths': (2, 5, 10, 20)},
  {'coeff': 2, 'w': 2, 'widths': (2, 5, 10, 20)},
  {'coeff': 2, 'w': 5, 'widths': (2, 5, 

The `ComprehensiveFCParameters` are the biggest set of features. It will take the longest to calculate

In [13]:
settings_comprehensive = ComprehensiveFCParameters()
settings_comprehensive

{'abs_energy': None,
 'absolute_sum_of_changes': None,
 'approximate_entropy': [{'m': 2, 'r': 0.1},
  {'m': 2, 'r': 0.3},
  {'m': 2, 'r': 0.5},
  {'m': 2, 'r': 0.7},
  {'m': 2, 'r': 0.9}],
 'ar_coefficient': [{'coeff': 0, 'k': 10},
  {'coeff': 1, 'k': 10},
  {'coeff': 2, 'k': 10},
  {'coeff': 3, 'k': 10},
  {'coeff': 4, 'k': 10}],
 'augmented_dickey_fuller': None,
 'autocorrelation': [{'lag': 0},
  {'lag': 1},
  {'lag': 2},
  {'lag': 3},
  {'lag': 4},
  {'lag': 5},
  {'lag': 6},
  {'lag': 7},
  {'lag': 8},
  {'lag': 9}],
 'binned_entropy': [{'max_bins': 10}],
 'count_above_mean': None,
 'count_below_mean': None,
 'cwt_coefficients': [{'coeff': 0, 'w': 2, 'widths': (2, 5, 10, 20)},
  {'coeff': 0, 'w': 5, 'widths': (2, 5, 10, 20)},
  {'coeff': 0, 'w': 10, 'widths': (2, 5, 10, 20)},
  {'coeff': 0, 'w': 20, 'widths': (2, 5, 10, 20)},
  {'coeff': 1, 'w': 2, 'widths': (2, 5, 10, 20)},
  {'coeff': 1, 'w': 5, 'widths': (2, 5, 10, 20)},
  {'coeff': 1, 'w': 10, 'widths': (2, 5, 10, 20)},
  {'coe

You see those parameters as values in the fc_paramter dictionary? Those are the parameters of the feature extraction methods.

In detail, the value in a fc_parameters dicitonary can contain a list of dictionaries. Every dictionary in that list is one feature.

So, for example

In [18]:
settings_comprehensive['large_number_of_peaks']

[{'n': 1}, {'n': 3}, {'n': 5}]

would trigger the calculation of 3 different "large_number_of_peaks" features, one for n=1, for n=3 and finally for n=5.  Lets just take them and extract some features

In [19]:
settings_value_count = {'large_number_of_peaks': settings_comprehensive['large_number_of_peaks']}
settings_value_count

{'large_number_of_peaks': [{'n': 1}, {'n': 3}, {'n': 5}]}

In [20]:
X_tsfresh = extract_features(df, column_id="id", default_fc_parameters=settings_value_count)
X_tsfresh.head()

Feature Extraction: 100%|██████████| 2/2 [00:00<00:00, 130.93it/s]


,pressure__large_number_of_peaks__n_3,pressure__large_number_of_peaks__n_1,pressure__large_number_of_peaks__n_5,temperature__large_number_of_peaks__n_5,temperature__large_number_of_peaks__n_3,temperature__large_number_of_peaks__n_1
id,,,,,,
a,0.0,0.0,0.0,0.0,0.0,0.0
b,0.0,0.0,0.0,0.0,0.0,0.0


The nice thing is, we actually contain the parameters in the feature name, so it is possible to reconstruct 
how the feature was calculated.

In [21]:
from_columns(X_tsfresh)

{'pressure': {'large_number_of_peaks': [{'n': 3}, {'n': 1}, {'n': 5}]},
 'temperature': {'large_number_of_peaks': [{'n': 5}, {'n': 3}, {'n': 1}]}}

This means that you should never change a column name. Otherwise the information how it was calculated can get lost.